# Hubert Stefanski - Data Mining II - Main Assignment
## 20081102@mail.wit.ie

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder
from sklearn import model_selection
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

from IPython.display import display, Markdown
plt.style.use("seaborn-darkgrid")
pd.set_option('display.max_columns', None)  

import os
for d in ['data','output']: os.makedirs(d, exist_ok=True)

## Helper functions and globals

In [2]:

def name_func(df):
    col_names = []
    for i in range(df.shape[1]):
        col_names.append("Col_" + str(i))
    return col_names

DEBUG = False
SEED = 64
TARGET = "Target"

## Load Datasets

In [3]:
# Load and name columns, change last column name to "Target"
df = pd.read_csv("data/labeled.csv.gz",names = name_func(pd.read_csv("data/labeled.csv.gz")))
df.columns = [*df.columns[:-1],TARGET]
print("Labeled", df.shape)
# df_labeled.head()

# Load and name columns
df_scoring = pd.read_csv("data/scoring.csv.gz",names = name_func(pd.read_csv("data/scoring.csv.gz")))
print("Scoring", df_scoring.shape)
# df_scoring.head()
                         

Labeled (16000, 177)
Scoring (9000, 176)


### Preliminary analysis

In [4]:
df.shape

(16000, 177)

In [5]:
df.head(5)

,Col_0,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,Col_9,Col_10,Col_11,Col_12,Col_13,Col_14,Col_15,Col_16,Col_17,Col_18,Col_19,Col_20,Col_21,Col_22,Col_23,Col_24,Col_25,Col_26,Col_27,Col_28,Col_29,Col_30,Col_31,Col_32,Col_33,Col_34,Col_35,Col_36,Col_37,Col_38,Col_39,Col_40,Col_41,Col_42,Col_43,Col_44,Col_45,Col_46,Col_47,Col_48,Col_49,Col_50,Col_51,Col_52,Col_53,Col_54,Col_55,Col_56,Col_57,Col_58,Col_59,Col_60,Col_61,Col_62,Col_63,Col_64,Col_65,Col_66,Col_67,Col_68,Col_69,Col_70,Col_71,Col_72,Col_73,Col_74,Col_75,Col_76,Col_77,Col_78,Col_79,Col_80,Col_81,Col_82,Col_83,Col_84,Col_85,Col_86,Col_87,Col_88,Col_89,Col_90,Col_91,Col_92,Col_93,Col_94,Col_95,Col_96,Col_97,Col_98,Col_99,Col_100,Col_101,Col_102,Col_103,Col_104,Col_105,Col_106,Col_107,Col_108,Col_109,Col_110,Col_111,Col_112,Col_113,Col_114,Col_115,Col_116,Col_117,Col_118,Col_119,Col_120,Col_121,Col_122,Col_123,Col_124,Col_125,Col_126,Col_127,Col_128,Col_129,Col_130,Col_131,Col_132,Col_133,Col_134,Col_135,Col_136,Col_137,Col_138,Col_139,Col_140,Col_141,Col_142,Col_143,Col_144,Col_145,Col_146,Col_147,Col_148,Col_149,Col_150,Col_151,Col_152,Col_153,Col_154,Col_155,Col_156,Col_157,Col_158,Col_159,Col_160,Col_161,Col_162,Col_163,Col_164,Col_165,Col_166,Col_167,Col_168,Col_169,Col_170,Col_171,Col_172,Col_173,Col_174,Col_175,Target
0,127.2792,2.1250,0.0089,0.000000,0,1.0856,0.8347,2,67.0820,1.0901,0.0133,6,1.2049,0.0,4956.4503,700.3571,0.900,3,0.63,0.1467,5667.93,3.0673,1.0,0.000140,84.8528,yes,0.0507,0.0,yes,3.0,0.002192,0.54,0.00,0.00,0.00,0.001548,0.0,0.2970,3.0,18.6663,no,no,3.15,0.000072,0.0,2,1.26,0.0844,19.1795,26.1972,67.082,1,0.2041,1.1347,0.0000,2005,6.3,0.0044,1861.1658,0.0,1.1258,0.003336,84.8528,3,0.0000,0.0867,19.1707,1.1214,1.62,0.00028,0.0,12.0810,0.024,0.0,no,84.8528,0.031492,15951.33,0.81,2.3,0.000028,3,0.4329,917.48,0.0,0.0592,0.0453,0.0,141.3693,0.0489,0.90,40.6461,2.1111,6.7931,NaN,0.0000,yes,0.708948,1,2.9167,0.0,0.0000,1.1001,0.0,4,1.0699,0.0000,1.3556,8.3191,1.1098,11.070,0.000,708.57,0.1022,1.0847,no,0,4.5882,1.8910,75.06,34.83,0.251908,0.000,6,0.2053,2,1.5000,0.00,7.8250,11.7621,1.1214,907.9648,40.2680,34.1960,1.0333,0.0,0.0,0.0000,1.5600,914,3.0,49.32,0.810,0.0000,0.000000,0.0,0.0000,1.26,0.0900,0.000056,yes,5.0,0.00,0.0,0.990,1152.5624,2.0,4,0.63,1.0427,no,0.0000,0.0,0.0222,0.0900,0.0,0.0089,0.63,YES,0.0,1.0858,0.0,3.0,0.0000,0.0000,5,0
1,90.0000,27.8151,0.8178,0.000024,0,1.1216,0.0453,2,84.8528,0.0000,0.1867,5,1.1206,0.0,184.5735,4410.4081,0.675,2,27.45,0.2533,15250.41,0.0000,1.0,0.009880,60.0000,yes,0.1653,0.0,no,3.0,0.000000,0.00,0.54,0.54,0.81,0.000060,0.0,23.9751,3.0,19.6452,no,no,222.30,0.000264,0.0,2,0.00,0.0000,19.1315,17.9053,67.082,1,0.0000,1.1289,1.5414,2004,0.0,0.0844,205.1430,0.0,1.1141,0.000396,60.0000,2,1.9359,3.7533,18.7933,1.0945,5.94,0.00000,0.0,1.0206,0.788,0.0,yes,0.0000,0.033176,5908.32,1.17,0.6,0.001220,3,0.4702,1269.60,0.0,0.0696,8.7413,0.0,745.9945,0.0000,0.99,38.7104,3.6471,5.1000,5.0,0.0000,yes,0.262592,1,14.3000,0.0,1.0831,1.0892,0.0,4,0.0000,0.0187,0.3333,0.0000,1.1337,1.035,28.260,746.46,0.0089,1.1046,yes,0,0.0000,0.0450,8.91,1.35,0.677796,67.082,5,0.0000,2,8.5143,14.22,2.1250,40.0762,1.1141,0.0000,26.2787,37.5227,0.2720,0.0,0.0,155.7775,0.7111,1272,3.0,0.00,1.125,0.0356,0.000632,0.0,0.0044,314.10,0.2004,0.013960,yes,5.0,0.00,0.0,0.675,212.1320,2.0,3,0.72,0.0000,no,4.6538,1.4,0.0267,3.8032,0.0,0.0222,0.90,yes,0.0,1.1100,0.0,3.0,1.0747,0.3227,5,0
2,0.0000,57.5814,2.7556,0.000000,0,1.1581,0.3973,2,60.0000,0.0000,0.0000,5,1.2339,0.0,67.0820,120.0000,0.900,2,0.00,0.2044,12095.64,0.0000,1.0,0.000000,90.0000,yes,0.0000,0.0,yes,3.0,0.000000,0.00,0.00,0.00,0.00,0.000740,0.0,0.0000,3.0,21.9022,no,no,0.00,0.000000,0.0,2,0.00,0.0000,44.3260,52.6417,67.082,1,0.0000,1.1899,0.0000,2002,0.0,0.0000,113.9297,0.0,1.1355,0.039532,60.0000,2,0.0000,0.0000,15.7787,1.1283,0.00,0.00000,0.0,46.9860,0.000,0.0,no,0.0000,0.023440,2248.65,0.00,0.7,0.000000,3,0.6096,1381.98,0.0,0.0710,83.1013,0.0,261.7626,0.0000,1.35,38.5000,0.0000,22.5879,5.0,0.0

In [6]:
df.dtypes.value_counts()

float64    149
int64       18
object      10
dtype: int64

In [7]:
df_na = df.isna().sum()
df_na[df_na > 0].index 


Index(['Col_22', 'Col_29', 'Col_38', 'Col_94', 'Col_140', 'Col_151', 'Col_156',
       'Col_172'],
      dtype='object')

In [8]:
df.select_dtypes("int64").nunique().sort_values()

Target        2
Col_7         4
Col_125       4
Col_51        4
Col_63        4
Col_175       4
Col_98        4
Col_55        5
Col_157       6
Col_123       6
Col_81        6
Col_45        6
Col_17        6
Col_104       6
Col_11        7
Col_116      14
Col_4        27
Col_139    1224
dtype: int64

### Explicitly cleanup Col_4 and Col_139, they have too many values to be simply categorical

In [9]:
df.Col_4 = df.Col_4.astype(float)
df.Col_139 = df.Col_139.astype(float)

Dataset is a mixture of categorical and numerical data, this will have to be cleaned up to exclude or encode categoricals.

The dataset also seems to have a lot of empty columns, It might be possible that they will affect the score, this will have to be checked again after building the model, then eventually removing columns to test the difference, no point doing so in the cleanup for now.
There are also missing values 

## Cleanup
Cleanup the dataset to mitigate the issues mentioned above, mainly:
- Encode categorical to binary flags
- Drop NaNs and Nulls

In [10]:
def cleanup(df):
    # Changing categorical data to a binary flag, 
    for col in df.select_dtypes(include=['object']).copy():
         # Ensure all strings are lower before encoding
        df[col] = df[col].str.lower()
        encoder = OrdinalEncoder(categories=[['no','yes']])
        df[col] = encoder.fit_transform(df.loc[:,[col]])

    # Check whether NaNs exist
    print("Nan exists?: " + str(df.isnull().values.any()))

    # Check total number of NaNs
    print("NaN count: " + str(df.isnull().sum().sum()))

    # Cleanup NaNs
    for col in df.columns:
        if df[col].isna().values.any():
            # Replace all NaNs with mode of the column
            df[col] = df[col].fillna(df[col].mode().values[0])

   
        # Check whether NaNs exist after nan Cleanup      
    print("Nan exists?: " + str(df.isnull().values.any()))
    # Check total number of NaNs
    print("NaN count: " + str(df.isnull().sum().sum()))
    
    return df
df = cleanup(df)

Nan exists?: True
NaN count: 3365
Nan exists?: False
NaN count: 0


### Post-Cleanup check
Datatypes were corrected in the cleanup() function, ensure that there is no categorical data left

In [11]:
df

,Col_0,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,Col_9,Col_10,Col_11,Col_12,Col_13,Col_14,Col_15,Col_16,Col_17,Col_18,Col_19,Col_20,Col_21,Col_22,Col_23,Col_24,Col_25,Col_26,Col_27,Col_28,Col_29,Col_30,Col_31,Col_32,Col_33,Col_34,Col_35,Col_36,Col_37,Col_38,Col_39,Col_40,Col_41,Col_42,Col_43,Col_44,Col_45,Col_46,Col_47,Col_48,Col_49,Col_50,Col_51,Col_52,Col_53,Col_54,Col_55,Col_56,Col_57,Col_58,Col_59,Col_60,Col_61,Col_62,Col_63,Col_64,Col_65,Col_66,Col_67,Col_68,Col_69,Col_70,Col_71,Col_72,Col_73,Col_74,Col_75,Col_76,Col_77,Col_78,Col_79,Col_80,Col_81,Col_82,Col_83,Col_84,Col_85,Col_86,Col_87,Col_88,Col_89,Col_90,Col_91,Col_92,Col_93,Col_94,Col_95,Col_96,Col_97,Col_98,Col_99,Col_100,Col_101,Col_102,Col_103,Col_104,Col_105,Col_106,Col_107,Col_108,Col_109,Col_110,Col_111,Col_112,Col_113,Col_114,Col_115,Col_116,Col_117,Col_118,Col_119,Col_120,Col_121,Col_122,Col_123,Col_124,Col_125,Col_126,Col_127,Col_128,Col_129,Col_130,Col_131,Col_132,Col_133,Col_134,Col_135,Col_136,Col_137,Col_138,Col_139,Col_140,Col_141,Col_142,Col_143,Col_144,Col_145,Col_146,Col_147,Col_148,Col_149,Col_150,Col_151,Col_152,Col_153,Col_154,Col_155,Col_156,Col_157,Col_158,Col_159,Col_160,Col_161,Col_162,Col_163,Col_164,Col_165,Col_166,Col_167,Col_168,Col_169,Col_170,Col_171,Col_172,Col_173,Col_174,Col_175,Target
0,127.2792,2.1250,0.0089,0.000000,0.0,1.0856,0.8347,2,67.0820,1.0901,0.0133,6,1.2049,0.0000,4956.4503,700.3571,0.900,3,0.63,0.1467,5667.93,3.0673,1.0,0.000140,84.8528,1.0,0.0507,0.0,1.0,3.0,0.002192,0.54,0.000,0.00,0.00,0.001548,0.000000,0.2970,3.0,18.6663,0.0,0.0,3.15,0.000072,0.0000,2,1.260,0.0844,19.1795,26.1972,67.0820,1,0.2041,1.1347,0.0000,2005,6.30,0.0044,1861.1658,0.0,1.1258,0.003336,84.8528,3,0.0000,0.0867,19.1707,1.1214,1.62,0.000280,0.0,12.0810,0.024,0.0000,0.0,84.8528,0.031492,15951.33,0.81,2.3,0.000028,3,0.4329,917.48,0.0,0.0592,0.0453,0.00,141.3693,0.0489,0.90,40.6461,2.1111,6.7931,5.0,0.0000,1.0,0.708948,1,2.9167,0.0000,0.0000,1.1001,0.000,4,1.0699,0.0000,1.3556,8.3191,1.1098,11.070,0.000,708.57,0.1022,1.0847,0.0,0,4.5882,1.8910,75.06,34.83,0.251908,0.000,6,0.2053,2,1.5000,0.00,7.8250,11.7621,1.1214,907.9648,40.2680,34.1960,1.0333,0.0000,0.0,0.0000,1.5600,914.0,3.0,49.32,0.810,0.0000,0.000000,0.0000,0.0000,1.26,0.0900,0.000056,1.0,5.0,0.0000,0.0,0.990,1152.5624,2.0,4,0.63,1.0427,0.0,0.0000,0.0000,0.0222,0.0900,0.0,0.0089,0.63,1.0,0.0,1.0858,0.0,3.0,0.0000,0.0000,5,0
1,90.0000,27.8151,0.8178,0.000024,0.0,1.1216,0.0453,2,84.8528,0.0000,0.1867,5,1.1206,0.0000,184.5735,4410.4081,0.675,2,27.45,0.2533,15250.41,0.0000,1.0,0.009880,60.0000,1.0,0.1653,0.0,0.0,3.0,0.000000,0.00,0.540,0.54,0.81,0.000060,0.000000,23.9751,3.0,19.6452,0.0,0.0,222.30,0.000264,0.0000,2,0.000,0.0000,19.1315,17.9053,67.0820,1,0.0000,1.1289,1.5414,2004,0.00,0.0844,205.1430,0.0,1.1141,0.000396,60.0000,2,1.9359,3.7533,18.7933,1.0945,5.94,0.000000,0.0,1.0206,0.788,0.0000,1.0,0.0000,0.033176,5908.32,1.17,0.6,0.001220,3,0.4702,1269.60,0.0,0.0696,8.7413,0.00,745.9945,0.0000,0.99,38.7104,3.6471,5.1000,5.0,0.0000,1.0,0.262592,1,14.3000,0.0000,1.0831,1.0892,0.000,4,0.0000,0.0187,0.3333,0.0000,1.1337,1.035,28.260,746.46,0.0089,1.1046,1.0,0,0.0000,0.0450,8.91,1.35,0.677796,67.082,5,0.0000,2,8.5143,14.22,2.1250,40.0762,1.1141,0.0000,26.2787,37.5227,0.2720,0.0000,0.0,155.7775,0.7111,1272.0,3.0,0.00,1.125,0.0356,0.000632,0.0000,0.0044,314.10,0.2004,0.013960,1.0,5.0,0.0000,0.0,0.675,212.1320,2.0,3,0.72,0.0000,0.0,4.6538,1.4000,0.0267,3.8032,0.0,0.0222,0.90,1.0,0.0,1.1100,0.0,3.0,1.0747,0.3227,5,0
2,0.0000,57.5814,2.7556,0.000000,0.0,1.1581,0.3973,2,60.0000,0.0000,0.0000,5,1.2339,0.0000,67.0820,120.0000,0.900,2,0.00,0.2044,12095.64,0.0000,1.0,0.000000,90.0000,1.0,0.0000,0.0,1.0,3.0,0.000000,0.00,0.000,0.00,0.00,0.000740,0.000000,0.0000,3.0,21.9022,0.0,0.0,0.00,0.000000,0.0000,2,0.000,0.0000,44.3260,52.6417,67.0820,1,0.0000,1.1899,0.0000,2002,0.00,0.0000,113.9297,0.0,1.1355,0.039532,60.0000,2,0.0000,0.0000,15.7787,1.1283,0.00,0.000000,0.0,46.9860,0.000,0.0000,0.0,0.0000,0.023440,2248

In [12]:
df.dtypes.value_counts()

float64    161
int64       16
dtype: int64

In [13]:
df_na = df.isna().sum()
df_na[df_na > 0].index 

Index([], dtype='object')

No more categoricals or na, nice.

In [14]:
df.shape

(16000, 177)

In [15]:
df.head(5)

,Col_0,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,Col_9,Col_10,Col_11,Col_12,Col_13,Col_14,Col_15,Col_16,Col_17,Col_18,Col_19,Col_20,Col_21,Col_22,Col_23,Col_24,Col_25,Col_26,Col_27,Col_28,Col_29,Col_30,Col_31,Col_32,Col_33,Col_34,Col_35,Col_36,Col_37,Col_38,Col_39,Col_40,Col_41,Col_42,Col_43,Col_44,Col_45,Col_46,Col_47,Col_48,Col_49,Col_50,Col_51,Col_52,Col_53,Col_54,Col_55,Col_56,Col_57,Col_58,Col_59,Col_60,Col_61,Col_62,Col_63,Col_64,Col_65,Col_66,Col_67,Col_68,Col_69,Col_70,Col_71,Col_72,Col_73,Col_74,Col_75,Col_76,Col_77,Col_78,Col_79,Col_80,Col_81,Col_82,Col_83,Col_84,Col_85,Col_86,Col_87,Col_88,Col_89,Col_90,Col_91,Col_92,Col_93,Col_94,Col_95,Col_96,Col_97,Col_98,Col_99,Col_100,Col_101,Col_102,Col_103,Col_104,Col_105,Col_106,Col_107,Col_108,Col_109,Col_110,Col_111,Col_112,Col_113,Col_114,Col_115,Col_116,Col_117,Col_118,Col_119,Col_120,Col_121,Col_122,Col_123,Col_124,Col_125,Col_126,Col_127,Col_128,Col_129,Col_130,Col_131,Col_132,Col_133,Col_134,Col_135,Col_136,Col_137,Col_138,Col_139,Col_140,Col_141,Col_142,Col_143,Col_144,Col_145,Col_146,Col_147,Col_148,Col_149,Col_150,Col_151,Col_152,Col_153,Col_154,Col_155,Col_156,Col_157,Col_158,Col_159,Col_160,Col_161,Col_162,Col_163,Col_164,Col_165,Col_166,Col_167,Col_168,Col_169,Col_170,Col_171,Col_172,Col_173,Col_174,Col_175,Target
0,127.2792,2.1250,0.0089,0.000000,0.0,1.0856,0.8347,2,67.0820,1.0901,0.0133,6,1.2049,0.0,4956.4503,700.3571,0.900,3,0.63,0.1467,5667.93,3.0673,1.0,0.000140,84.8528,1.0,0.0507,0.0,1.0,3.0,0.002192,0.54,0.00,0.00,0.00,0.001548,0.0,0.2970,3.0,18.6663,0.0,0.0,3.15,0.000072,0.0,2,1.26,0.0844,19.1795,26.1972,67.082,1,0.2041,1.1347,0.0000,2005,6.3,0.0044,1861.1658,0.0,1.1258,0.003336,84.8528,3,0.0000,0.0867,19.1707,1.1214,1.62,0.00028,0.0,12.0810,0.024,0.0,0.0,84.8528,0.031492,15951.33,0.81,2.3,0.000028,3,0.4329,917.48,0.0,0.0592,0.0453,0.0,141.3693,0.0489,0.90,40.6461,2.1111,6.7931,5.0,0.0000,1.0,0.708948,1,2.9167,0.0,0.0000,1.1001,0.0,4,1.0699,0.0000,1.3556,8.3191,1.1098,11.070,0.000,708.57,0.1022,1.0847,0.0,0,4.5882,1.8910,75.06,34.83,0.251908,0.000,6,0.2053,2,1.5000,0.00,7.8250,11.7621,1.1214,907.9648,40.2680,34.1960,1.0333,0.0,0.0,0.0000,1.5600,914.0,3.0,49.32,0.810,0.0000,0.000000,0.0,0.0000,1.26,0.0900,0.000056,1.0,5.0,0.00,0.0,0.990,1152.5624,2.0,4,0.63,1.0427,0.0,0.0000,0.0,0.0222,0.0900,0.0,0.0089,0.63,1.0,0.0,1.0858,0.0,3.0,0.0000,0.0000,5,0
1,90.0000,27.8151,0.8178,0.000024,0.0,1.1216,0.0453,2,84.8528,0.0000,0.1867,5,1.1206,0.0,184.5735,4410.4081,0.675,2,27.45,0.2533,15250.41,0.0000,1.0,0.009880,60.0000,1.0,0.1653,0.0,0.0,3.0,0.000000,0.00,0.54,0.54,0.81,0.000060,0.0,23.9751,3.0,19.6452,0.0,0.0,222.30,0.000264,0.0,2,0.00,0.0000,19.1315,17.9053,67.082,1,0.0000,1.1289,1.5414,2004,0.0,0.0844,205.1430,0.0,1.1141,0.000396,60.0000,2,1.9359,3.7533,18.7933,1.0945,5.94,0.00000,0.0,1.0206,0.788,0.0,1.0,0.0000,0.033176,5908.32,1.17,0.6,0.001220,3,0.4702,1269.60,0.0,0.0696,8.7413,0.0,745.9945,0.0000,0.99,38.7104,3.6471,5.1000,5.0,0.0000,1.0,0.262592,1,14.3000,0.0,1.0831,1.0892,0.0,4,0.0000,0.0187,0.3333,0.0000,1.1337,1.035,28.260,746.46,0.0089,1.1046,1.0,0,0.0000,0.0450,8.91,1.35,0.677796,67.082,5,0.0000,2,8.5143,14.22,2.1250,40.0762,1.1141,0.0000,26.2787,37.5227,0.2720,0.0,0.0,155.7775,0.7111,1272.0,3.0,0.00,1.125,0.0356,0.000632,0.0,0.0044,314.10,0.2004,0.013960,1.0,5.0,0.00,0.0,0.675,212.1320,2.0,3,0.72,0.0000,0.0,4.6538,1.4,0.0267,3.8032,0.0,0.0222,0.90,1.0,0.0,1.1100,0.0,3.0,1.0747,0.3227,5,0
2,0.0000,57.5814,2.7556,0.000000,0.0,1.1581,0.3973,2,60.0000,0.0000,0.0000,5,1.2339,0.0,67.0820,120.0000,0.900,2,0.00,0.2044,12095.64,0.0000,1.0,0.000000,90.0000,1.0,0.0000,0.0,1.0,3.0,0.000000,0.00,0.00,0.00,0.00,0.000740,0.0,0.0000,3.0,21.9022,0.0,0.0,0.00,0.000000,0.0,2,0.00,0.0000,44.3260,52.6417,67.082,1,0.0000,1.1899,0.0000,2002,0.0,0.0000,113.9297,0.0,1.1355,0.039532,60.0000,2,0.0000,0.0000,15.7787,1.1283,0.00,0.00000,0.0,46.9860,0.000,0.0,0.0,0.0000,0.023440,2248.65,0.00,0.7,0.000000,3,0.6096,1381.98,0.0,0.0710,83.1013,0.0,261.7626,0.0000,1.35,38.5000,

In [16]:
## Checking for null values
missing_vals = df.isnull().values.sum()
print("Missing values: ", missing_vals)

Missing values:  0


#### Comment:
no need to fix dtypes, all int64 and float64 and no missing values

### Categorical/Numerical split

In [17]:
# Get column names as list
attributes = df.columns.tolist()
# Remove the target column, no point in doing this if we include it
attributes.remove(TARGET)

temp = df[attributes].nunique()

categorical = list(temp[(temp>1)& (temp<20)].index)

numerical = list(temp[(temp>20)].index)

f"Categorical count: {len(categorical)} and Numerical count:{len(numerical)}"

'Categorical count: 29 and Numerical count:143'

## Output the cleaned up DataFrame to a fresh csv
This allows some insight through dTale

In [18]:
# df.to_csv("output/clean_df_labeled.csv", compression='gzip' ,index=False)

### Train-Test split

In [19]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, stratify=df[TARGET], test_size=.40, random_state=SEED)

## Checking for null values
missing_vals = df.isnull().values.sum()
print("Missing values: ", missing_vals)
print(df_train.shape,df_test.shape)

Missing values:  0
(9600, 177) (6400, 177)


In [20]:
df_train.head(1)

,Col_0,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,Col_9,Col_10,Col_11,Col_12,Col_13,Col_14,Col_15,Col_16,Col_17,Col_18,Col_19,Col_20,Col_21,Col_22,Col_23,Col_24,Col_25,Col_26,Col_27,Col_28,Col_29,Col_30,Col_31,Col_32,Col_33,Col_34,Col_35,Col_36,Col_37,Col_38,Col_39,Col_40,Col_41,Col_42,Col_43,Col_44,Col_45,Col_46,Col_47,Col_48,Col_49,Col_50,Col_51,Col_52,Col_53,Col_54,Col_55,Col_56,Col_57,Col_58,Col_59,Col_60,Col_61,Col_62,Col_63,Col_64,Col_65,Col_66,Col_67,Col_68,Col_69,Col_70,Col_71,Col_72,Col_73,Col_74,Col_75,Col_76,Col_77,Col_78,Col_79,Col_80,Col_81,Col_82,Col_83,Col_84,Col_85,Col_86,Col_87,Col_88,Col_89,Col_90,Col_91,Col_92,Col_93,Col_94,Col_95,Col_96,Col_97,Col_98,Col_99,Col_100,Col_101,Col_102,Col_103,Col_104,Col_105,Col_106,Col_107,Col_108,Col_109,Col_110,Col_111,Col_112,Col_113,Col_114,Col_115,Col_116,Col_117,Col_118,Col_119,Col_120,Col_121,Col_122,Col_123,Col_124,Col_125,Col_126,Col_127,Col_128,Col_129,Col_130,Col_131,Col_132,Col_133,Col_134,Col_135,Col_136,Col_137,Col_138,Col_139,Col_140,Col_141,Col_142,Col_143,Col_144,Col_145,Col_146,Col_147,Col_148,Col_149,Col_150,Col_151,Col_152,Col_153,Col_154,Col_155,Col_156,Col_157,Col_158,Col_159,Col_160,Col_161,Col_162,Col_163,Col_164,Col_165,Col_166,Col_167,Col_168,Col_169,Col_170,Col_171,Col_172,Col_173,Col_174,Col_175,Target
10116,60.0,22.7931,0.6356,0.000516,0.0,1.1447,0.1093,3,94.8683,0.0,0.0889,6,1.1989,0.0,60.0,4467.684,0.63,2,29.7,0.1289,12277.62,0.0,1.0,0.002672,90.0,1.0,0.5813,0.00002,0.0,3.0,0.0,0.63,0.81,11.61,0.0,0.000168,0.0,6.2984,3.0,27.3705,0.0,0.0,60.12,0.001472,0.0,3,0.0,0.0,24.669,26.1543,67.082,1,0.0424,1.1564,1.7731,2005,1.8,0.0756,838.4903,0.0,1.134,0.007164,67.082,2,0.6653,1.1493,19.98,1.1258,33.12,0.00008,0.45,15.698,0.7707,0.0,1.0,0.0,0.02838,9108.9,2.43,4.1,0.00132,4,0.5374,1407.35,0.0044,0.0657,5.28,0.0,425.192,0.0133,0.99,44.5266,5.5897,10.3882,5.0,1374.1179,1.0,0.40484,1,8.7115,0.0,1.048,1.064,0.0,3,1.064,0.224,1.0711,0.0,1.1327,11.7,34.38,638.55,0.0267,1.0861,1.0,0,2.5556,0.2012,161.19,3.78,0.545672,67.082,5,0.0613,2,8.1351,6.03,3.2308,6.4568,1.1176,5590.7155,45.688,49.9053,2.328,0.0,0.45,60.0,1.6222,1402.0,3.0,0.0,0.9,0.0222,0.000268,0.0,0.04,167.13,4.3777,0.007428,1.0,4.0,0.9525,0.0,0.72,60.0,1.0,3,0.72,0.0,0.0,3.3529,3.6522,0.0311,1.509,1.0699,0.0311,1.26,1.0,1.2,1.0993,0.016,2.0,1.1065,0.1413,4,0


### Feature Engineering

In [21]:



def feature_engineering(df_train,df_test,debug=True):
    ohe = preprocessing.OneHotEncoder(sparse=False, handle_unknown='ignore')
    df_train_model = pd.DataFrame()
    df_test_model = pd.DataFrame()

    
    for feature in categorical:
        #train
        if debug:
            print("Categorical Features")
            print(f"Train Feature {feature} with {df_train[feature].nunique()} levels")
        X = ohe.fit_transform(df_train[feature].values.reshape(-1,1))
        df_tmp = pd.DataFrame(X, columns = [f"{feature}_{k}" for k in range(X.shape[1])])
        df_train_model = pd.concat([df_train_model, df_tmp], axis=1)

        #test
        X = ohe.transform(df_test[feature].values.reshape(-1,1))
        df_tmp = pd.DataFrame(X, columns = [f"{feature}_{k}" for k in range(X.shape[1])])
        df_test_model = pd.concat([df_test_model, df_tmp], axis=1)
        
    for feature in numerical:
        if debug:
            print(f"\tFeature {feature}")
        #train
        X =df_train[feature].values
        df_train_model[feature]=X
        df_train_model[f"{feature}_is_zero"] = (X==0).astype("int64")
        df_train_model[f"{feature}_log"] = np.log(X+1)

        #test
        X =df_test[feature].values
        df_test_model[feature]=X
        df_test_model[f"{feature}_is_zero"] = (X==0).astype("int64")
        df_test_model[f"{feature}_log"] = np.log(X+1)
        
        # Add the target back in
        
    df_train_model['Target'] = df_train['Target'].values
    if 'Target' in df_test.columns:
        df_test_model['Target'] = df_test['Target'].values

    print("Train dataset", df_train_model.shape)
    print("Test dataset", df_test_model.shape)

    return df_train_model, df_test_model

df_train_model, df_test_model = feature_engineering(df_train,df_test,debug=False)
 

Train dataset (9600, 559)
Test dataset (6400, 559)


## Model Building

#### Data normalization and scaling

In [22]:
from sklearn.preprocessing import StandardScaler

features = df_train_model.columns[:-1].tolist()
print("Num of features", len(features))

X_train, y_train = df_train_model[features], df_train_model[TARGET]
X_test, y_test = df_test_model[features],df_test_model[TARGET]

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Num of features 558


In [23]:

model = RandomForestClassifier(criterion='entropy', max_depth=4)

score = model_selection.cross_val_score(model, X_train, y_train, cv=10, n_jobs=-1, scoring='accuracy')


In [24]:
print(f"Accuracy CV mean score is {score.mean():.2%} with a std of {score.std():.2%}")

Accuracy CV mean score is 71.95% with a std of 0.84%


### Feature Selection

In [25]:
models = []

from sklearn.linear_model import LogisticRegression
models.append( {"name": "LogisticRegression", "ref": LogisticRegression(solver="liblinear", max_iter=2000)} )

from sklearn.ensemble import RandomForestClassifier
models.append( {"name": "RFC ", "ref": RandomForestClassifier(criterion='entropy', max_depth=4)} )

from sklearn.naive_bayes import GaussianNB
models.append({"name": "GNB ", "ref": GaussianNB()})

print("Models:")
for model in models:
    print(f"    {model['name']:30} {model['ref']}")

Models:
    LogisticRegression             LogisticRegression(max_iter=2000, solver='liblinear')
    RFC                            RandomForestClassifier(criterion='entropy', max_depth=4)
    GNB                            GaussianNB()


In [26]:
# select_features = np.array(features)[rfecv.support_].tolist()


# select_features = np.array(numerical[:57])
# select_features

In [27]:
select_features = ['Col_17_0', 'Col_17_1', 'Col_17_2', 'Col_17_3', 'Col_17_4', 'Col_17_5', 'Col_22_0', 'Col_22_1', 'Col_22_2', 'Col_22_3', 'Col_25_0', 'Col_25_1', 'Col_28_0', 'Col_28_1', 'Col_29_0', 'Col_29_1', 'Col_29_2', 'Col_29_3', 'Col_38_0', 'Col_38_1', 'Col_38_2', 'Col_38_3', 'Col_40_0', 'Col_40_1', 'Col_45_0', 'Col_45_1', 'Col_45_2', 'Col_45_3', 'Col_45_4', 'Col_45_5', 'Col_51_0', 'Col_51_1', 'Col_51_2', 'Col_51_3', 'Col_55_0', 'Col_55_1', 'Col_55_2', 'Col_55_3', 'Col_55_4', 'Col_63_0', 'Col_63_1', 'Col_63_2', 'Col_63_3', 'Col_74_0', 'Col_74_1', 'Col_81_0', 'Col_81_1', 'Col_81_2', 'Col_81_3', 'Col_81_4', 'Col_81_5', 'Col_94_0', 'Col_94_1', 'Col_94_2', 'Col_94_3', 'Col_98_0', 'Col_98_1', 'Col_98_2', 'Col_98_3', 'Col_104_0', 'Col_104_1', 'Col_104_2', 'Col_104_3', 'Col_104_4', 'Col_104_5', 'Col_115_0', 'Col_115_1', 'Col_116_0', 'Col_116_1', 'Col_116_2', 'Col_116_3', 'Col_116_4', 'Col_116_5', 'Col_116_6', 'Col_116_7', 'Col_116_8', 'Col_116_10', 'Col_116_11', 'Col_116_12', 'Col_123_0', 'Col_123_2', 'Col_123_3', 'Col_123_4', 'Col_123_5', 'Col_125_0', 'Col_125_1', 'Col_125_3', 'Col_140_0', 'Col_140_1', 'Col_140_2', 'Col_140_3', 'Col_151_0', 'Col_156_0', 'Col_156_1', 'Col_156_2', 'Col_157_1', 'Col_157_3', 'Col_157_5', 'Col_168_1', 'Col_172_0', 'Col_172_2', 'Col_172_3', 'Col_175_0', 'Col_175_1', 'Col_175_2', 'Col_175_3', 'Col_0', 'Col_0_log', 'Col_1', 'Col_1_log', 'Col_2', 'Col_2_log', 'Col_3', 'Col_3_is_zero', 'Col_3_log', 'Col_4_is_zero', 'Col_4_log', 'Col_5', 'Col_5_log', 'Col_6', 'Col_6_log', 'Col_8', 'Col_8_log', 'Col_9', 'Col_10', 'Col_10_is_zero', 'Col_10_log', 'Col_12', 'Col_12_log', 'Col_13', 'Col_13_log', 'Col_14', 'Col_14_log', 'Col_15', 'Col_15_log', 'Col_16', 'Col_16_is_zero', 'Col_16_log', 'Col_18', 'Col_19', 'Col_19_log', 'Col_20', 'Col_20_is_zero', 'Col_20_log', 'Col_21', 'Col_21_is_zero', 'Col_21_log', 'Col_23', 'Col_23_log', 'Col_24_log', 'Col_26', 'Col_26_is_zero', 'Col_26_log', 'Col_30', 'Col_30_log', 'Col_31_log', 'Col_32_log', 'Col_33', 'Col_33_log', 'Col_34_log', 'Col_35', 'Col_35_log', 'Col_36', 'Col_36_log', 'Col_37', 'Col_37_log', 'Col_39', 'Col_39_log', 'Col_42', 'Col_42_log', 'Col_43', 'Col_43_log', 'Col_44', 'Col_44_log', 'Col_46', 'Col_46_is_zero', 'Col_46_log', 'Col_47', 'Col_47_log', 'Col_48', 'Col_48_log', 'Col_49', 'Col_49_log', 'Col_50', 'Col_50_log', 'Col_52_is_zero', 'Col_52_log', 'Col_53', 'Col_53_is_zero', 'Col_53_log', 'Col_54_is_zero', 'Col_56', 'Col_56_is_zero', 'Col_56_log', 'Col_57', 'Col_60', 'Col_61', 'Col_65', 'Col_65_is_zero', 'Col_65_log', 'Col_66', 'Col_66_is_zero', 'Col_66_log', 'Col_67', 'Col_67_is_zero', 'Col_67_log', 'Col_68', 'Col_68_is_zero', 'Col_68_log', 'Col_69', 'Col_69_is_zero', 'Col_69_log', 'Col_70', 'Col_70_log', 'Col_71', 'Col_71_is_zero', 'Col_71_log', 'Col_72', 'Col_73', 'Col_73_log', 'Col_75', 'Col_75_log', 'Col_76', 'Col_76_is_zero', 'Col_76_log', 'Col_77', 'Col_77_is_zero', 'Col_77_log', 'Col_78', 'Col_78_log', 'Col_79', 'Col_79_is_zero', 'Col_79_log', 'Col_80', 'Col_80_log', 'Col_82', 'Col_82_log', 'Col_83', 'Col_83_log', 'Col_84', 'Col_84_log', 'Col_85', 'Col_85_log', 'Col_86', 'Col_86_is_zero', 'Col_86_log', 'Col_87', 'Col_87_is_zero', 'Col_87_log', 'Col_88', 'Col_88_log', 'Col_89', 'Col_89_is_zero', 'Col_89_log', 'Col_90', 'Col_90_is_zero', 'Col_90_log', 'Col_91', 'Col_91_log', 'Col_92', 'Col_92_log', 'Col_93', 'Col_93_log', 'Col_95', 'Col_95_log', 'Col_97', 'Col_97_log', 'Col_99', 'Col_99_is_zero', 'Col_99_log', 'Col_100', 'Col_100_is_zero', 'Col_100_log', 'Col_101', 'Col_101_is_zero', 'Col_102', 'Col_102_log', 'Col_103', 'Col_103_is_zero', 'Col_103_log', 'Col_105', 'Col_105_is_zero', 'Col_105_log', 'Col_106', 'Col_106_log', 'Col_107', 'Col_107_log', 'Col_108', 'Col_108_is_zero', 'Col_108_log', 'Col_109', 'Col_109_log', 'Col_110', 'Col_110_log', 'Col_111', 'Col_111_is_zero', 'Col_111_log', 'Col_112', 'Col_112_log', 'Col_113', 'Col_113_log', 'Col_114', 'Col_114_log', 'Col_117', 'Col_117_log', 'Col_118', 'Col_118_log', 'Col_119', 'Col_119_log', 'Col_120', 'Col_120_log', 'Col_121', 'Col_121_log', 'Col_122', 'Col_122_is_zero', 'Col_122_log', 'Col_124', 'Col_124_log', 'Col_126', 'Col_126_log', 'Col_127', 'Col_127_is_zero', 'Col_127_log', 'Col_128', 'Col_128_log', 'Col_129', 'Col_129_is_zero', 'Col_129_log', 'Col_130_log', 'Col_131', 'Col_131_is_zero', 'Col_131_log', 'Col_132', 'Col_132_log', 'Col_133', 'Col_133_log', 'Col_134', 'Col_135', 'Col_136', 'Col_137_log', 'Col_138', 'Col_138_log', 'Col_139', 'Col_139_log', 'Col_141', 'Col_141_log', 'Col_142', 'Col_142_log', 'Col_143_log', 'Col_144_is_zero', 'Col_145', 'Col_145_is_zero', 'Col_145_log', 'Col_146', 'Col_146_log', 'Col_147', 'Col_147_is_zero', 'Col_147_log', 'Col_148', 'Col_148_log', 'Col_149', 'Col_149_log', 'Col_152', 'Col_152_log', 'Col_153_log', 'Col_154', 'Col_154_log', 'Col_155', 'Col_155_log', 'Col_158', 'Col_158_log', 'Col_159', 'Col_159_log', 'Col_161_is_zero', 'Col_162_is_zero', 'Col_162_log', 'Col_163', 'Col_163_is_zero', 'Col_163_log', 'Col_164', 'Col_164_is_zero', 'Col_164_log', 'Col_165', 'Col_165_log', 'Col_166', 'Col_166_is_zero', 'Col_166_log']

### Re-split Train-Test

In [28]:
df_train_model_selected_features = df_train_model.loc[:,select_features + [TARGET]]
df_test_model_selected_features = df_test_model.loc[:,select_features + [TARGET]]

In [29]:
X_train = scaler.fit_transform(df_train_model.loc[:, select_features])
y_train = df_train_model.Target.values
X_test = scaler.transform(df_test_model.loc[:, select_features])
y_test = df_test_model.Target.values

In [30]:
for model in models:
    crossval_scores = model_selection.cross_val_score(model["ref"], X_train, y_train, scoring='accuracy', cv=5)
    print ("\nModel %s" % model["name"])
    print ("\tCV roc_auc scores %s" % [f"{s:.4f}" for s in crossval_scores])
    print ("\tmean=%.4f std=%.4f" % (crossval_scores.mean(), crossval_scores.std()))


Model LogisticRegression
	CV roc_auc scores ['0.6771', '0.6979', '0.7099', '0.6958', '0.6922']
	mean=0.6946 std=0.0106

Model RFC 
	CV roc_auc scores ['0.7156', '0.7240', '0.7214', '0.7266', '0.7255']
	mean=0.7226 std=0.0039

Model GNB 
	CV roc_auc scores ['0.4911', '0.5271', '0.4885', '0.4953', '0.5047']
	mean=0.5014 std=0.0140


### Hyper Parameter Optimisation
Going to use RandomForestClassifier, found this to be yielding the best mean scores, so it's probably best to focus on optimising this one.

### ROC Optimisation

In [31]:
# from time import process_time

# t1_start = process_time() 
# print(f"start")

# model = RandomForestClassifier(max_features= 'auto', random_state=SEED)
# # search_space = dict(criterion= ['entropy'],
# #                     bootstrap= [True], 
# #                     max_depth= [25,30,50],
# #                     min_samples_leaf= [1, 2, 4, 5, 6],
# #                     min_samples_split= [5,6,9,12],
# #                     max_features=['auto'],
# #                     n_estimators= [50, 150, 200],
# #                     warm_start=[True])

# search = model_selection.RandomizedSearchCV(model,search_space, random_state=SEED,scoring ="roc_auc")
# search_results = search.fit(X_train, y_train)

# t1_stop = process_time()
# print("Elapsed time:", t1_stop, t1_start) 
   
# print("Elapsed time during the whole program in seconds:",
#                                          t1_stop-t1_start) 


In [32]:
# print("Best: %f using %s" % (search_results.best_score_, search_results.best_params_))

### Accuracy Optimisation

In [33]:
# from time import process_time

# t1_start = process_time() 
# print(f"start")

# model = RandomForestClassifier(max_features= 'auto', random_state=SEED)

# # search_space = dict(criterion= ['entropy'],
# #                     bootstrap= [True], 
# #                     max_depth= [50,100,125],
# #                     min_samples_leaf= [1,3,5],
# #                     min_samples_split= [2,3,5],
# #                     max_features=['auto'],
# #                     n_estimators= [75,100,125],
# #                     warm_start=[True])

# search = model_selection.RandomizedSearchCV(model,search_space, random_state=SEED,scoring ="accuracy")
# search_results = search.fit(X_train, y_train)

# t1_stop = process_time()
# print("Elapsed time:", t1_stop, t1_start) 
   
# print("Elapsed time during the whole program in seconds:",
#                                          t1_stop-t1_start) 


In [34]:
# print("Best: %f using %s" % (search_results.best_score_, search_results.best_params_))

### Scoring -- `accuracy`

In [35]:
df_labeled = pd.read_csv("data/labeled.csv.gz",names = name_func(pd.read_csv("data/labeled.csv.gz")))
df_labeled.columns = [*df_labeled.columns[:-1],"Target"]

df_labeled = cleanup(df_labeled)
print(df_labeled.shape)
df_labeled.head(1)
df_labeled.dtypes.value_counts()
df_na = df_labeled.isna().sum()


Nan exists?: True
NaN count: 3365
Nan exists?: False
NaN count: 0
(16000, 177)


In [36]:
df_scoring = pd.read_csv("data/scoring.csv.gz", names = name_func(pd.read_csv("data/scoring.csv.gz")))

df_scoring = cleanup(df_scoring)
print(df_scoring.shape)
df_scoring.head(1)
df_scoring.dtypes.value_counts()
df_na = df_scoring.isna().sum()


Nan exists?: True
NaN count: 1856
Nan exists?: False
NaN count: 0
(9000, 176)


In [37]:
df_train_model, df_test_model = feature_engineering(df_labeled,df_scoring,False)

Train dataset (16000, 560)
Test dataset (9000, 559)


In [38]:
X_train, y_train = df_train_model[select_features], df_train_model['Target']
X_test = df_test_model[select_features]

In [39]:
df_train_model_select_features = df_train_model.loc[:, select_features + [TARGET]]

In [40]:
df_test_model_select_features = df_test_model.loc[:, select_features]

In [41]:
X_train = scaler.fit_transform(df_train_model_select_features.loc[:,select_features])
y_train = df_train_model_select_features.Target.values
X_test = scaler.transform(df_test_model_select_features.loc[:,select_features])

In [42]:

ss = preprocessing.StandardScaler()
df_train_model_scaled = ss.fit_transform(df_train_model)
df_test_model_scaled = ss.transform(df_train_model)

In [43]:
X_train, y_train = df_train_model[select_features], df_train_model['Target']
X_test = df_test_model[select_features]

In [44]:
model = RandomForestClassifier(warm_start=True,criterion='entropy', max_features= 'auto', random_state=SEED,n_estimators= 75, min_samples_split= 5, min_samples_leaf= 8, max_depth= 125, bootstrap= True)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

In [45]:
pd.DataFrame(data=y_pred, columns=['target']).to_csv("pred_accuracy.csv.gz",compression="gzip", index=False)

### Scoring -- `roc_auc`

In [46]:
model = RandomForestClassifier(warm_start= True,criterion='entropy', max_features= 'auto', random_state=SEED,n_estimators= 200, min_samples_split= 9, min_samples_leaf= 6, max_depth= 50, bootstrap= True)
model.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', max_depth=50, min_samples_leaf=6,
                       min_samples_split=9, n_estimators=200, random_state=64,
                       warm_start=True)

In [47]:
y_pr = model.predict_proba(X_test)[:,0]

In [48]:
pd.DataFrame(data = y_pr, columns=['pr_target']).to_csv("pred_roc_auc.csv.gz", compression = "gzip", index=False)